# Exercise 3: Shape Reconstruction

**Submission Deadline**: 09.06.2021, 23:55

We will take a look at two major approaches for 3D shape reconstruction in this last exercise.

Like in exercise 2, you can run all trainings either locally or on Google Colab. Just follow the instructions below. 

Note that training reconstruction methods generally takes relatively long, even for simple shape completion. Training the generalization will take a few hours. *Thus, please make sure to start training well before the submission deadline.*

## 3.0. Running this notebook
We recommend running this notebook on a cuda compatible local gpu. You can also run training on cpu, it will just take longer.

We describe two options for executing the training parts of this exercise below: Using Google Colab or running it locally on your machine. If you are not planning on using Colab, just skip forward to Local Execution.

### Google Colab

If you don't have access to gpu and don't wish to train on CPU, you can use Google Colab. However, we experienced the issue that inline visualization of shapes or inline images didn't work on colab, so just keep that in mind.
What you can also do is only train networks on colab, download the checkpoint, and visualize inference locally.

In case you're using Google Colab, you can upload the exercise folder (containing `exercise_3.ipynb`, directory `exercise_3` and the file `requirements.txt`) as `3d-machine-learning` to google drive (make sure you don't upload extracted datasets files).
Additionally you'd need to open the notebook `exercise_3.ipynb` in Colab using `File > Open Notebook > Upload`.

Next you'll need to run these two cells for setting up the environment. Before you do that make sure your instance has a GPU.

In [1]:
'''
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# We assume you uploaded the exercise folder in root Google Drive folder

!cp -r /content/drive/MyDrive/3d-machine-learning 3d-machine-learning/
os.chdir('/content/3d-machine-learning/')
print('Installing requirements')
!pip install -r requirements.txt
'''
# Make sure you restart runtime when directed by Colab

"\nimport os\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n\n# We assume you uploaded the exercise folder in root Google Drive folder\n\n!cp -r /content/drive/MyDrive/3d-machine-learning 3d-machine-learning/\nos.chdir('/content/3d-machine-learning/')\nprint('Installing requirements')\n!pip install -r requirements.txt\n"

Run this cell after restarting your colab runtime

In [2]:
'''
import os
import sys
import torch
os.chdir('/content/3d-machine-learning/')
sys.path.insert(1, "/content/3d-machine-learning/")
print('CUDA availability:', torch.cuda.is_available())
'''

'\nimport os\nimport sys\nimport torch\nos.chdir(\'/content/3d-machine-learning/\')\nsys.path.insert(1, "/content/3d-machine-learning/")\nprint(\'CUDA availability:\', torch.cuda.is_available())\n'

### Local Execution

If you run this notebook locally, you have to first install the python dependiencies again. They are the same as for exercise 1 so you can re-use the environment you used last time. If you use [poetry](https://python-poetry.org), you can also simply re-install everything (`poetry install`) and then run this notebook via `poetry run jupyter notebook`.

### Imports

The following imports should work regardless of whether you are using Colab or local execution.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

Use the next cell to test whether a GPU was detected by pytorch.

In [2]:
torch.cuda.is_available()

True

## 3.1 Shape Reconstruction from 3D SDF grids with 3D-EPN

In the first part of this exercise, we will take a look at shape complation using [3D-EPN](https://arxiv.org/abs/1612.00101). This approach was also introduced in the lecture.

The visualization below shows an overview of the method: From an incomplete shape observation (which you would get when scanning an object with a depth sensor for example), we use a 3D encoder-predictor network that first encodes the incomplete shapes into a common latent space using several 3D convolution layers and then decodes them again using multiple 3D transpose convolutions.

This way, we get from a 32^3 SDF voxel grid to a 32^3 DF (unsigned) voxel grid that represents the completed shape. We only focus on this part here; in the original implementation, this 32^3 completed prediction would then be further improved (in an offline step after inference) by sampling parts from a shape database to get the final resolution to 128^3.

<img src="exercise_3/images/3depn_teaser.png" alt="3D-EPN Teaser" style="width: 800px;"/>

The next steps will follow the structure we established in exercise 2: Taking a look at the dataset structure and downloading the data; then, implementing dataset, model, and training loop.

### (a) Downloading the data
We will use the original dataset used in the official implementation. It consists of SDF and DF grids (representing incomplete input data and complete target data) with a resolution of 32^3 each. Each input-target pair is generated from a ShapeNet shape.

The incomplete SDF data are generated by sampling virtual camera trajectories around every object. Each trajectory is assigned an ID which is part of the file names (see below). The camera views for each trajectory are combined into a common SDF grid by volumetric fusion. It is easy to generate an SDF here since we know both camera location and object surface: Everything between camera and surface is known free space and outside the object, leading to a positive SDF sign. Everything behind the surface has a negative sign. For the complete shapes, however, deciding whether a voxel in the DF grid is inside or outside an object is not a trivial problem. This is why we use unsigned distance fields as target and prediction representation instead. This still encodes the distance to the closest surface but does not contain explicit information about the inside/outside location.

In terms of dataset layout, we follow the ShapeNet directory structure as seen in the last exercise:
Each folder in the `exercise_3/data/shapenet_dim32_sdf` and `exercise_3/data/shapenet_dim32_df` directories contains one shape category represented by a number, e.g. `02691156`.
We provide the mapping between these numbers and the corresponding names in `exercise_3/data/shape_info.json`. Each of these shape category folders contains lots of shapes in sdf or df format. In addition to that, every shape now also contains multiple trajectories: 0 to 7, encoded as `__0__` to `__7__`. These 8 files are just different input representations, meaning they vary in the level of completeness and location of missing parts; they all map to the `.df` file with corresponding shape ID and `__0__` at the end.

```
# contents of exercise_2/data/shapenet_dim32_sdf
02691156/                                           # Shape category folder with all its shapes
    ├── 10155655850468db78d106ce0a280f87__0__.sdf   # Trajectory 0 for a shape of the category
    ├── 10155655850468db78d106ce0a280f87__1__.sdf   # Trajectory 1 for the same shape
    ├── :                                      
    ├── 10155655850468db78d106ce0a280f87__7__.sdf   # Trajectory 7 for the same shape
    ├── 1021a0914a7207aff927ed529ad90a11__0__.sdf   # Trajectory 0 for another shape
    ├── :                                           # And so on ...
02933112/                                           # Another shape category folder
02958343/                                           # In total you should have 8 shape category folders
:

# contents of exercise_2/data/shapenet_dim32_df
02691156/                                           # Shape category folder with all its shapes
    ├── 10155655850468db78d106ce0a280f87__0__.df    # A single shape of the category
    ├── 1021a0914a7207aff927ed529ad90a11__0__.df    # Another shape of the category
    ├── :                                           # And so on ...
02933112/                                           # Another shape category folder
02958343/                                           # In total you should have 55 shape category folders
:
```

Download and extract the data with the code cell below.

In [5]:
'''
print('Downloading ...')
# File sizes: 11GB for shapenet_dim32_sdf.zip (incomplete scans), 4GB for shapenet_dim32_df.zip (target shapes)
!wget http://kaldir.vc.in.tum.de/adai/CNNComplete/shapenet_dim32_sdf.zip -P ~/datasets/e3
!wget http://kaldir.vc.in.tum.de/adai/CNNComplete/shapenet_dim32_df.zip -P ~/datasets/e3
print('Extracting ...')
!unzip -q ~/datasets/e3/shapenet_dim32_sdf.zip -d ~/datasets/e3
!unzip -q ~/datasets/e3/shapenet_dim32_df.zip -d ~/datasets/e3
!rm ~/datasets/e3/shapenet_dim32_sdf.zip
!rm ~/datasets/e3/shapenet_dim32_df.zip
print('Done.')
'''

"\nprint('Downloading ...')\n# File sizes: 11GB for shapenet_dim32_sdf.zip (incomplete scans), 4GB for shapenet_dim32_df.zip (target shapes)\n!wget http://kaldir.vc.in.tum.de/adai/CNNComplete/shapenet_dim32_sdf.zip -P ~/datasets/e3\n!wget http://kaldir.vc.in.tum.de/adai/CNNComplete/shapenet_dim32_df.zip -P ~/datasets/e3\nprint('Extracting ...')\n!unzip -q ~/datasets/e3/shapenet_dim32_sdf.zip -d ~/datasets/e3\n!unzip -q ~/datasets/e3/shapenet_dim32_df.zip -d ~/datasets/e3\n!rm ~/datasets/e3/shapenet_dim32_sdf.zip\n!rm ~/datasets/e3/shapenet_dim32_df.zip\nprint('Done.')\n"

### (b) Dataset

The dataset implementation follows the same general structure as in exercise 2. We prepared an initial implementation already in `exercise_3/data/shapenet.py`; your task is to resolve all TODOs there.

The data for SDFs and DFs in `.sdf`/`.df` files are stored in binary form as follows:
```
dimX    #uint64 
dimY    #uint64 
dimZ    #uint64 
data    #(dimX*dimY*dimZ) floats for sdf/df values
```
The SDF values stored per-voxel represent the distance to the closest surface *in voxels*.

You have to take care of three important steps before returning the SDF and DF for the corresponding `index` in `__getitem__`:
1. **Truncation**: 3D-EPN uses a truncated SDF which means that for each voxel, the distance to the closest surface will be clamped to a max absolute value. This is helpful since we do not care about longer distances (Marching Cubes only cares about distances close to the surface). It allows us to focus our predictions on the voxels near the surface. We use a `truncation_distance` of 3 (voxels) which means we expect to get an SDF with values between -3 and 3 as input to the model.
2. **Separation** of distances and sign: 3D-EPN uses as input a 2x32x32x32 SDF grid, with absolute distance values of the SDF in channel 0 and the signs (-1 or 1) in channel 1.
3. **Log** scaling: We scale targets and prediction with a log operation to further guide predictions to focus on the surface voxels. Therefore, you should return target DFs as `log(df + 1)`.

**Hint**: An easy way to load the data from `.sdf` and `.df` files is to use `np.fromfile`. First, load the dimensions, then the data, then reshape everything into the shape you loaded in the beginning. Make sure you get the datatypes and byte offsets right!

In [3]:
from exercise_3.data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train')
val_dataset = ShapeNet('val')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(val_dataset)}')  # expected output: 32304
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 153540
Length of val set: 32304
Length of overfit set: 64


In [4]:
# Visualize some shapes
from exercise_3.util.visualization import visualize_mesh
from skimage.measure import marching_cubes

train_sample = train_dataset[1]
print(f'Name: {train_sample["name"]}')  # expected output: 03001627/798a46965d9e0edfcea003eff0268278__3__-03001627/798a46965d9e0edfcea003eff0268278__0__
print(f'Input SDF: {train_sample["input_sdf"].shape}')  # expected output: (2, 32, 32, 32)
print(f'Target DF: {train_sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(train_sample['input_sdf'][0], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Name: 03001627/798a46965d9e0edfcea003eff0268278__3__-03001627/798a46965d9e0edfcea003eff0268278__0__
Input SDF: (2, 32, 32, 32)
Target DF: (32, 32, 32)


Output()

In [8]:
'''
train_sample = train_dataset[223]
print(f'Name: {train_sample["name"]}')  # expected output: 04379243/a1be21c9a71d133dc5beea20858a99d5__5__-04379243/a1be21c9a71d133dc5beea20858a99d5__0__
print(f'Input SDF: {train_sample["input_sdf"].shape}')  # expected output: (2, 32, 32, 32)
print(f'Target DF: {train_sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(train_sample['input_sdf'][0], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
'''

'\ntrain_sample = train_dataset[223]\nprint(f\'Name: {train_sample["name"]}\')  # expected output: 04379243/a1be21c9a71d133dc5beea20858a99d5__5__-04379243/a1be21c9a71d133dc5beea20858a99d5__0__\nprint(f\'Input SDF: {train_sample["input_sdf"].shape}\')  # expected output: (2, 32, 32, 32)\nprint(f\'Target DF: {train_sample["target_df"].shape}\')  # expected output: (32, 32, 32)\n\ninput_mesh = marching_cubes(train_sample[\'input_sdf\'][0], level=1)\nvisualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)\n'

In [9]:
'''
train_sample = train_dataset[95]
print(f'Name: {train_sample["name"]}')  # expected output: 03636649/3889631e42a84b0f51f77a6d7299806__2__-03636649/3889631e42a84b0f51f77a6d7299806__0__
print(f'Input SDF: {train_sample["input_sdf"].shape}')  # expected output: (2, 32, 32, 32)
print(f'Target DF: {train_sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(train_sample['input_sdf'][0], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
'''

'\ntrain_sample = train_dataset[95]\nprint(f\'Name: {train_sample["name"]}\')  # expected output: 03636649/3889631e42a84b0f51f77a6d7299806__2__-03636649/3889631e42a84b0f51f77a6d7299806__0__\nprint(f\'Input SDF: {train_sample["input_sdf"].shape}\')  # expected output: (2, 32, 32, 32)\nprint(f\'Target DF: {train_sample["target_df"].shape}\')  # expected output: (32, 32, 32)\n\ninput_mesh = marching_cubes(train_sample[\'input_sdf\'][0], level=1)\nvisualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)\n'

### (c) Model

The model architecture of 3D-EPN is visualized below:

<img src="exercise_3/images/3depn.png" alt="3D-EPN Architecture" style="width: 800px;"/>

For this exercise, we simplify the model by omitting the classification part - this will not have a big impact since most of the shape completion performance comes from the 3D encoder-decoder unet.

The model consists of three parts: The encoder, the bottleneck, and the decoder. Encoder and decoder are constructed with the same architecture, just mirrored.

The details of each part are:
- **Encoder**: 4 layers, each one containing a 3D convolution (with kernel size 4, as seen in the visualization), a 3D batch norm (except the very first layer), and a leaky ReLU with a negative slope of 0.2. Our goal is to reduce the spatial dimension from 32x32x32 to 1x1x1 and to get the feature dimension from 2 (absolute values and sign) to `num_features * 8`. We do this by using a stride of 2 and padding of 1 for all convolutions except for the last one where we use a stride of 1 and no padding. The feature channels are increased from 2 to `num_features` in the first layer and then doubled with every subsequent layer.
- **Decoder**: Same architecture as encoder, just mirrored: Going from `num_features * 8 * 2` (the 2 will be explained later) to 1 (the DF values). The spatial dimensions go from 1x1x1 to 32x32x32. Each layer use a 3D Transpose convolution now, together with 3D batch norm and ReLU (no leaky ReLUs anymore). Note that the last layer uses neither Batch Norms nor a ReLU since we do not want to constrain the range of possible values for the prediction.
- **Bottleneck**: This is realized with 2 fully connected layers, each one going from a vector of size 640 (which is `num_features * 8`) to a vector of size 640. Each such layer is followed by a ReLU activation.

Some minor details:
- **Skip connections** allow the decoder to use information from the encoder and also improve gradient flow. We use it here to connect the output of encoder layer 1 to decoder layer 4, the output of encoder layer 2 to decoder layer 3, and so on. This means that the input to a decoder layer is the concatenation of the previous decoder output with the corresponding encoder output, along the feature dimension. Hence, the number of input features for each decoder layer are twice those of the encoder layers, as mentioned above.
- **Log scaling**: You also need to scale the final outputs of the network logarithmically: `out = log(out + 1)`. This is the same transformation you applied to the target shapes in the dataloader before and ensures that prediction and target volumes are comparable.

With this in mind, implement the network architecture and `forward()` function in `exercise_3/model/threedepn.py`. You can check your architecture with the cell below.

In [5]:
from exercise_3.model.threedepn import ThreeDEPN
from exercise_3.util.model import summarize_model

threedepn = ThreeDEPN()
print(summarize_model(threedepn))  # Expected: Rows 0-34 and TOTAL = 52455681

sdf = torch.randn(4, 1, 32, 32, 32) * 2. - 1.
input_tensor = torch.cat([torch.abs(sdf), torch.sign(sdf)], dim=1)
predictions = threedepn(input_tensor)

print('Output tensor shape: ', predictions.shape)  # Expected: torch.Size([4, 32, 32, 32])

   | Name         | Type            | Params  
----------------------------------------------------
0  | conv1        | Conv3d          | 10320   
1  | conv2        | Conv3d          | 819360  
2  | conv3        | Conv3d          | 3277120 
3  | conv4        | Conv3d          | 13107840
4  | bn1          | BatchNorm3d     | 320     
5  | bn2          | BatchNorm3d     | 640     
6  | bn3          | BatchNorm3d     | 1280    
7  | leaky_relu   | LeakyReLU       | 0       
8  | bottleneck   | Sequential      | 820480  
9  | bottleneck.0 | Linear          | 410240  
10 | bottleneck.1 | ReLU            | 0       
11 | bottleneck.2 | Linear          | 410240  
12 | bottleneck.3 | ReLU            | 0       
13 | tconv1       | ConvTranspose3d | 26214720
14 | tconv2       | ConvTranspose3d | 6553760 
15 | tconv3       | ConvTranspose3d | 1638480 
16 | tconv4       | ConvTranspose3d | 10241   
17 | bn4          | BatchNorm3d     | 640     
18 | bn5          | BatchNorm3d     | 320     
19 | bn

### (d) Training script and overfitting to a single shape reconstruction

You can now go to the train script in `exercise_3/training/train_3depn.py` and fill in the missing pieces as you did for exercise 2. Then, verify that your training work by overfitting to a few samples below.

In [12]:
from exercise_3.training import train_3depn
config = {
    'experiment_name': '3_1_3depn_overfitting',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate': 0.001,
    'max_epochs': 250,
    'print_every_n': 10,
    'validate_every_n': 25,
}
#train_3depn.main(config)  # should be able to get <0.0025 train_loss and <0.13 val_loss

Using device: cuda:0
[004/00001] train_loss: 0.070432
[009/00001] train_loss: 0.022295
[012/00000] val_loss: 0.527366 | best_loss_val: 0.527366
[014/00001] train_loss: 0.013221
[019/00001] train_loss: 0.008615
[024/00001] train_loss: 0.006580
[024/00001] val_loss: 0.203785 | best_loss_val: 0.203785
[029/00001] train_loss: 0.005581
[034/00001] train_loss: 0.004817
[037/00000] val_loss: 0.159729 | best_loss_val: 0.159729
[039/00001] train_loss: 0.004245
[044/00001] train_loss: 0.003877
[049/00001] train_loss: 0.003666
[049/00001] val_loss: 0.142577 | best_loss_val: 0.142577
[054/00001] train_loss: 0.003449
[059/00001] train_loss: 0.003286
[062/00000] val_loss: 0.131892 | best_loss_val: 0.131892
[064/00001] train_loss: 0.003148
[069/00001] train_loss: 0.003061
[074/00001] train_loss: 0.002986
[074/00001] val_loss: 0.126460 | best_loss_val: 0.126460
[079/00001] train_loss: 0.002917
[084/00001] train_loss: 0.002839
[087/00000] val_loss: 0.122761 | best_loss_val: 0.122761
[089/00001] train_l

### (e) Training over the entire training set
If the overfitting works, we can go ahead with training on the entire dataset.

**Note**: As is the case with most reconstruction networks and considering the size of the model (> 50M parameters), this training will take a few hours on a GPU. *Please make sure to start training early enough before the submission deadline.*

In [13]:
config = {
    'experiment_name': '3_1_3depn_generalization',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate': 0.001,
    'max_epochs': 5,
    'print_every_n': 50,
    'validate_every_n': 1000,
}
#train_3depn.main(config)  # should be able to get best_loss_val < 0.1 after a few hours and 5 epochs

Using device: cuda:0
[000/00049] train_loss: 0.033006
[000/00099] train_loss: 0.014199
[000/00149] train_loss: 0.011504
[000/00199] train_loss: 0.010343
[000/00249] train_loss: 0.009146
[000/00299] train_loss: 0.008594
[000/00349] train_loss: 0.008901
[000/00399] train_loss: 0.008294
[000/00449] train_loss: 0.008185
[000/00499] train_loss: 0.007624
[000/00549] train_loss: 0.007329
[000/00599] train_loss: 0.007106
[000/00649] train_loss: 0.007092
[000/00699] train_loss: 0.007133
[000/00749] train_loss: 0.006871
[000/00799] train_loss: 0.006601
[000/00849] train_loss: 0.006673
[000/00899] train_loss: 0.006765
[000/00949] train_loss: 0.006230
[000/00999] train_loss: 0.006361
[000/00999] val_loss: 0.365919 | best_loss_val: 0.365919
[000/01049] train_loss: 0.006302
[000/01099] train_loss: 0.005775
[000/01149] train_loss: 0.006159
[000/01199] train_loss: 0.006223
[000/01249] train_loss: 0.006039
[000/01299] train_loss: 0.005752
[000/01349] train_loss: 0.005842
[000/01399] train_loss: 0.00559

[002/01901] train_loss: 0.003525
[002/01951] train_loss: 0.003596
[002/02001] train_loss: 0.003570
[002/02051] train_loss: 0.003497
[002/02101] train_loss: 0.003525
[002/02151] train_loss: 0.003466
[002/02201] train_loss: 0.003572
[002/02251] train_loss: 0.003199
[002/02301] train_loss: 0.003400
[002/02351] train_loss: 0.003530
[002/02401] train_loss: 0.003337
[002/02401] val_loss: 0.215063 | best_loss_val: 0.112165
[002/02451] train_loss: 0.003497
[002/02501] train_loss: 0.003403
[002/02551] train_loss: 0.003382
[002/02601] train_loss: 0.003385
[002/02651] train_loss: 0.003300
[002/02701] train_loss: 0.003205
[002/02751] train_loss: 0.003369
[002/02801] train_loss: 0.003430
[002/02851] train_loss: 0.003423
[002/02901] train_loss: 0.003450
[002/02951] train_loss: 0.003288
[002/03001] train_loss: 0.003474
[002/03051] train_loss: 0.003316
[002/03101] train_loss: 0.003530
[002/03151] train_loss: 0.003473
[002/03201] train_loss: 0.003279
[002/03251] train_loss: 0.003571
[002/03301] train_l

[004/03803] train_loss: 0.002726
[004/03803] val_loss: 0.123703 | best_loss_val: 0.097932
[004/03853] train_loss: 0.002607
[004/03903] train_loss: 0.002880
[004/03953] train_loss: 0.002847
[004/04003] train_loss: 0.002806
[004/04053] train_loss: 0.002759
[004/04103] train_loss: 0.002992
[004/04153] train_loss: 0.002830
[004/04203] train_loss: 0.002726
[004/04253] train_loss: 0.002940
[004/04303] train_loss: 0.002944
[004/04353] train_loss: 0.002801
[004/04403] train_loss: 0.002629
[004/04453] train_loss: 0.002714
[004/04503] train_loss: 0.002674
[004/04553] train_loss: 0.002633
[004/04603] train_loss: 0.002784
[004/04653] train_loss: 0.002696
[004/04703] train_loss: 0.002609
[004/04753] train_loss: 0.002687


### (f) Inference

Implement the missing bits in `exercise_3/inference/infer_3depn.py`. You should then be able to see your reconstructions below.

The outputs of our provided visualization functions are, from left to right:
- Input, partial shape
- Predicted completion
- Target shape

In [6]:
from exercise_3.util.visualization import visualize_meshes
from exercise_3.inference.infer_3depn import InferenceHandler3DEPN

# create a handler for inference using a trained checkpoint
inferer = InferenceHandler3DEPN('exercise_3/runs/3_1_3depn_generalization/model_best.ckpt')

In [27]:
input_sdf = ShapeNet.get_shape_sdf('03636649/b286c9c136784db2af1744fdb1fbe7df__0__')
target_df = ShapeNet.get_shape_df('03636649/b286c9c136784db2af1744fdb1fbe7df__0__')

input_mesh, reconstructed_mesh, target_mesh = inferer.infer_single(input_sdf, target_df)
visualize_meshes([input_mesh, reconstructed_mesh, target_mesh], flip_axes=True)

Output()

In [28]:
input_sdf = ShapeNet.get_shape_sdf('03636649/23eaba9bdd51a5b0dfe9cab879fd37e8__1__')
target_df = ShapeNet.get_shape_df('03636649/23eaba9bdd51a5b0dfe9cab879fd37e8__0__')

input_mesh, reconstructed_mesh, target_mesh = inferer.infer_single(input_sdf, target_df)
visualize_meshes([input_mesh, reconstructed_mesh, target_mesh], flip_axes=True)

Output()

In [29]:
input_sdf = ShapeNet.get_shape_sdf('02691156/5de2cc606b65b960e0b6546e08902f28__0__')
target_df = ShapeNet.get_shape_df('02691156/5de2cc606b65b960e0b6546e08902f28__0__')

input_mesh, reconstructed_mesh, target_mesh = inferer.infer_single(input_sdf, target_df)
visualize_meshes([input_mesh, reconstructed_mesh, target_mesh], flip_axes=True)

Output()

## 3.2 DeepSDF


Here, we will take a look at 3D-reconstruction using [DeepSDF](https://arxiv.org/abs/1901.05103). We recommend reading the paper before attempting the exercise.

DeepSDF is an auto-decoder based approach that learns a continuous SDF representation for a class of shapes. Once trained, it can be used for shape representation, interpolation and shape completion. We'll look at each of these
applications.

<img src="exercise_3/images/deepsdf_teaser.png" alt="deepsdf_teaser" style="width: 800px;"/>

During training, the autodecoder optimizes both the network parameters and the latent codes representing each of the training shapes. Once trained, to reconstruct a shape given its SDF observations, a latent code is
optimized keeping the network parameters fixed, such that the optimized latent code gives the lowest error with observed SDF values.

An advantage that implicit representations have over voxel/grid based approaches is that they are not tied to a particular grid resolution, and can be evaluated at any resolution once trained.

Similar to previous exercise, we'll first download the processed dataset, look at the implementation of the dataset, the model and the trainer, try out overfitting and generalization over the entire dataset, and finally inference on unseen samples.

### (a) Downloading the data

Whereas volumetric models output entire 3d shape representations, implicit models like DeepSDF work on per point basis. The network takes in a 3D-coordinate (and additionally the latent vector) and outputs the SDF value at the queried point. To train such a model,
we therefore need, for each of the training shapes, a bunch of points with their corresponding SDF values for supervision. Points are sampled more aggressively near the surface of the object as we want to capture a more detailed SDF near the surface. For those curious,
data preparation is decribed in more detail in section 5 of the paper.

We'll be using the ShapeNet Sofa class for the experiments in this exercise. We've already prepared this data, so that you don't need to deal with the preprocessing. For each shape, the following files are provided:
- `mesh.obj` representing the mesh representation of the shape
- `sdf.npz` file containing large number of points sampled on and around the mesh and their sdf values; contains numpy arrays under keys "pos" and "neg", containing points with positive and negative sdf values respectively

```
# contents of exercise_3/data/sdf_sofas
1faa4c299b93a3e5593ebeeedbff73b/                    # shape 0
    ├── mesh.obj                                    # shape 0 mesh
    ├── sdf.npz                                     # shape 0 sdf
    ├── surface.obj                                 # shape 0 surface
1fde48d83065ef5877a929f61fea4d0/                    # shape 1
1fe1411b6c8097acf008d8a3590fb522/                   # shape 2
:
```
Download and extract the data with the code cell below.

In [30]:
'''
print('Downloading ...')
# File sizes: ~10GB
!wget https://www.dropbox.com/s/4k5pw126nzus8ef/sdf_sofas.zip\?dl\=0 -O exercise_3/data/sdf_sofas.zip -P exercise_3/data

print('Extracting ...')
!unzip -q /usr/home/sut/datasets/e3/sdf_sofas.zip -d /usr/home/sut/datasets/e3
!rm /usr/home/sut/datasets/e3/sdf_sofas.zip

print('Done.')
'''

Extracting ...
Done.


### (b) Dataset

We provide a partial implementation of the dataset in `exercise_3/data/shape_implicit.py`.
Your task is to complete the `#TODOs` so that the dataset works as specified by the docstrings.

Once done, you can try running the following code blocks as sanity checks.

In [7]:
from exercise_3.data.shape_implicit import ShapeImplicit

num_points_to_samples = 40000
train_dataset = ShapeImplicit(num_points_to_samples, "train")
val_dataset = ShapeImplicit(num_points_to_samples, "val")
overfit_dataset = ShapeImplicit(num_points_to_samples, "overfit")

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 1226
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(val_dataset)}')  # expected output: 137
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 1

Length of train set: 1226
Length of val set: 137
Length of overfit set: 1


Let's take a look at the points sampled for a particular shape.

In [8]:
from exercise_3.util.visualization import visualize_mesh, visualize_pointcloud

shape_id = train_dataset[0]['name']
points = train_dataset[0]['points']
sdf = train_dataset[0]['sdf']

# sampled points inside the shape
inside_points = points[sdf[:, 0] < 0, :].numpy()

# sampled points outside the shape
outside_points = points[sdf[:, 0] > 0, :].numpy()

In [9]:
mesh = ShapeImplicit.get_mesh(shape_id)
print('Mesh')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

Mesh


/usr/home/sut/.conda/envs/idp/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [40]:
print('Sampled points with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)

Sampled points with negative SDF (inside)


Output()

In [41]:
print('Sampled points with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Sampled points with positive SDF (outside)


Output()

You'll notice that more points are sampled close to the surface rather than away from the surface.

### (c) Model

The DeepSDF auto-decoder architecture is visualized below:

<img src="exercise_3/images/deepsdf_architecture.png" alt="deepsdf_arch" style="width: 640px;"/>

Things to note:

- The network takes in the latent code for a shape concatenated with the query 3d coordinate, making up a 259 length vector (assuming latent code length is 256).
- The network consist of a sequence of weight-normed linear layers, each followed by a ReLU and a dropout. For weight norming a layer, check out `torch.nn.utils.weight_norm`. Each of these linear layers outputs a 512 dimensional vector, except the 4th layer which outputs a 253 dimensional vector.
- The output of the 4th layer is concatenated with the input, making the input to the 5th layer a 512 dimensional vector.
- The final layer is a simple linear layer without any norm, dropout or non-linearity, with a single dimensional output representing the SDF value.

Implement this architecture in file `exercise_3/model/deepsdf.py`.

Here are some basic sanity tests once you're done with your implementation.

In [12]:
from exercise_3.model.deepsdf import DeepSDFDecoder
from exercise_3.util.model import summarize_model

deepsdf = DeepSDFDecoder(latent_size=256)
print(summarize_model(deepsdf))

# input to the network is a concatenation of point coordinates (3) and the latent code (256 in this example);
# here we use a batch of 4096 points
input_tensor = torch.randn(4096, 3 + 256)
predictions = deepsdf(input_tensor)

print('\nOutput tensor shape: ', predictions.shape)  # expected output: 4096, 1

num_trainable_params = sum(p.numel() for p in deepsdf.parameters() if p.requires_grad) / 1e6
print(f'\nNumber of traininable params: {num_trainable_params:.2f}M')  # expected output: ~1.8M

   | Name    | Type           | Params 
---------------------------------------------
0  | wn1     | Linear         | 133632 
1  | wn2     | Linear         | 263168 
2  | wn3     | Linear         | 263168 
3  | wn4     | Linear         | 130042 
4  | wn5     | Linear         | 263168 
5  | wn6     | Linear         | 263168 
6  | wn7     | Linear         | 263168 
7  | wn8     | Linear         | 263168 
8  | fc      | Linear         | 513    
9  | relu    | ReLU           | 0      
10 | dropout | Dropout        | 0      
11 | TOTAL   | DeepSDFDecoder | 1843195

Output tensor shape:  torch.Size([4096, 1])

Number of traininable params: 1.84M


### (d) Training script and overfitting to a single shape

Fill in the train script in `exercise_3/training/train_deepsdf.py`, and verify that your training work by overfitting to a few samples below.

In [15]:
from exercise_3.training import train_deepsdf

overfit_config = {
    'experiment_name': '3_2_deepsdf_overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'num_sample_points': 4096,
    'latent_code_length': 256,
    'batch_size': 1,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,
    'print_every_n': 50,
    'visualize_every_n': 250,
}

train_deepsdf.main(overfit_config)  # expected loss around 0.0062

Using device: cuda:0
[049/00000] train_loss: 0.036574
[099/00000] train_loss: 0.024908
[149/00000] train_loss: 0.018325
[199/00000] train_loss: 0.014472
[249/00000] train_loss: 0.012499
[299/00000] train_loss: 0.011719
[349/00000] train_loss: 0.010442
[399/00000] train_loss: 0.010061
[449/00000] train_loss: 0.009350
[499/00000] train_loss: 0.009038
[549/00000] train_loss: 0.008423
[599/00000] train_loss: 0.008276
[649/00000] train_loss: 0.008126
[699/00000] train_loss: 0.007952
[749/00000] train_loss: 0.007851
[799/00000] train_loss: 0.007745
[849/00000] train_loss: 0.007636
[899/00000] train_loss: 0.007472
[949/00000] train_loss: 0.007446
[999/00000] train_loss: 0.007315
[1049/00000] train_loss: 0.007084
[1099/00000] train_loss: 0.007070
[1149/00000] train_loss: 0.006972
[1199/00000] train_loss: 0.006926
[1249/00000] train_loss: 0.006890
[1299/00000] train_loss: 0.006856
[1349/00000] train_loss: 0.006777
[1399/00000] train_loss: 0.006754
[1449/00000] train_loss: 0.006792
[1499/00000] 

Let's visualize the overfitted shape reconstruction to check if it looks reasonable.

In [16]:
# Load and visualize GT mesh of the overfit sample
gt_mesh = ShapeImplicit.get_mesh('7e728818848f191bee7d178666aae23d')
print('GT')
visualize_mesh(gt_mesh.vertices, gt_mesh.faces, flip_axes=True)

# Load and visualize reconstructed overfit sample; it's okay if they don't look visually exact, since we don't run 
# the training too long and have a learning rate decay while training 
mesh_path = "exercise_3/runs/3_2_deepsdf_overfit/meshes/01999_000.obj"
overfit_output = trimesh.load(mesh_path)
print('Overfit')
visualize_mesh(overfit_output.vertices, overfit_output.faces, flip_axes=True)

Output()

GT
Overfit


Output()

### (e) Training over entire train set

Once overfitting works, we can train on the entire train set.

Note: This training will take a few hours on a GPU (took ~3 hrs for 500 epochs on our 2080Ti, which already gave decent results). Please make sure to start training early enough before the submission deadline.

In [ ]:
from exercise_3.training import train_deepsdf

generalization_config = {
    'experiment_name': '3_2_deepsdf_generalization',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'num_sample_points': 4096, # you can adjust this such that the model fits on your gpu
    'latent_code_length': 256,
    'batch_size': 1,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,  # not necessary to run for 2000 epochs if you're short on time, at 500 epochs you should start to see reasonable results
    'print_every_n': 50,
    'visualize_every_n': 5000,
}

train_deepsdf.main(generalization_config)

Using device: cuda:0
[000/00049] train_loss: 0.036851
[000/00099] train_loss: 0.035113
[000/00149] train_loss: 0.035015
[000/00199] train_loss: 0.032840
[000/00249] train_loss: 0.033793
[000/00299] train_loss: 0.033049
[000/00349] train_loss: 0.032726
[000/00399] train_loss: 0.031962
[000/00449] train_loss: 0.031669
[000/00499] train_loss: 0.033765
[000/00549] train_loss: 0.033609
[000/00599] train_loss: 0.031352
[000/00649] train_loss: 0.032456
[000/00699] train_loss: 0.031511
[000/00749] train_loss: 0.031687
[000/00799] train_loss: 0.032332
[000/00849] train_loss: 0.032986
[000/00899] train_loss: 0.031316
[000/00949] train_loss: 0.030946
[000/00999] train_loss: 0.032103
[000/01049] train_loss: 0.031212
[000/01099] train_loss: 0.031363
[000/01149] train_loss: 0.030994
[000/01199] train_loss: 0.032314
[001/00023] train_loss: 0.031954
[001/00073] train_loss: 0.030850
[001/00123] train_loss: 0.032167
[001/00173] train_loss: 0.030934
[001/00223] train_loss: 0.030180
[001/00273] train_loss

[010/00189] train_loss: 0.022812
[010/00239] train_loss: 0.021524
[010/00289] train_loss: 0.021697
[010/00339] train_loss: 0.022148
[010/00389] train_loss: 0.021753
[010/00439] train_loss: 0.021383
[010/00489] train_loss: 0.020876
[010/00539] train_loss: 0.021279
[010/00589] train_loss: 0.021763
[010/00639] train_loss: 0.022156
[010/00689] train_loss: 0.022065
[010/00739] train_loss: 0.021402
[010/00789] train_loss: 0.021733
[010/00839] train_loss: 0.020511
[010/00889] train_loss: 0.021120
[010/00939] train_loss: 0.020994
[010/00989] train_loss: 0.022260
[010/01039] train_loss: 0.021153
[010/01089] train_loss: 0.021970
[010/01139] train_loss: 0.021376
[010/01189] train_loss: 0.021319
[011/00013] train_loss: 0.022439
[011/00063] train_loss: 0.023922
[011/00113] train_loss: 0.022925
[011/00163] train_loss: 0.020781
[011/00213] train_loss: 0.022753
[011/00263] train_loss: 0.021750
[011/00313] train_loss: 0.022196
[011/00363] train_loss: 0.021182
[011/00413] train_loss: 0.021775
[011/00463

[020/00379] train_loss: 0.019774
[020/00429] train_loss: 0.019528
[020/00479] train_loss: 0.018786
[020/00529] train_loss: 0.019236
[020/00579] train_loss: 0.019681
[020/00629] train_loss: 0.019782
[020/00679] train_loss: 0.018506
[020/00729] train_loss: 0.019079
[020/00779] train_loss: 0.018854
[020/00829] train_loss: 0.018454
[020/00879] train_loss: 0.018475
[020/00929] train_loss: 0.018866
[020/00979] train_loss: 0.019351
[020/01029] train_loss: 0.018185
[020/01079] train_loss: 0.018147
[020/01129] train_loss: 0.018895
[020/01179] train_loss: 0.019145
[021/00003] train_loss: 0.019358
[021/00053] train_loss: 0.021614
[021/00103] train_loss: 0.021599
[021/00153] train_loss: 0.020891
[021/00203] train_loss: 0.020321
[021/00253] train_loss: 0.020215
[021/00303] train_loss: 0.018401
[021/00353] train_loss: 0.018568
[021/00403] train_loss: 0.019549
[021/00453] train_loss: 0.017492
[021/00503] train_loss: 0.018883
[021/00553] train_loss: 0.019389
[021/00603] train_loss: 0.018209
[021/00653

[030/00569] train_loss: 0.018338
[030/00619] train_loss: 0.017723
[030/00669] train_loss: 0.017262
[030/00719] train_loss: 0.017956
[030/00769] train_loss: 0.017202
[030/00819] train_loss: 0.016946
[030/00869] train_loss: 0.018047
[030/00919] train_loss: 0.017746
[030/00969] train_loss: 0.017658
[030/01019] train_loss: 0.017764
[030/01069] train_loss: 0.017640
[030/01119] train_loss: 0.017771
[030/01169] train_loss: 0.017269
[030/01219] train_loss: 0.017887
[031/00043] train_loss: 0.021076
[031/00093] train_loss: 0.019368
[031/00143] train_loss: 0.019240
[031/00193] train_loss: 0.018646
[031/00243] train_loss: 0.018962
[031/00293] train_loss: 0.017781
[031/00343] train_loss: 0.017864
[031/00393] train_loss: 0.018322
[031/00443] train_loss: 0.018111
[031/00493] train_loss: 0.018086
[031/00543] train_loss: 0.017743
[031/00593] train_loss: 0.018185
[031/00643] train_loss: 0.017772
[031/00693] train_loss: 0.017374
[031/00743] train_loss: 0.016968
[031/00793] train_loss: 0.017491
[031/00843

[040/00759] train_loss: 0.016825
[040/00809] train_loss: 0.017373
[040/00859] train_loss: 0.017423
[040/00909] train_loss: 0.016908
[040/00959] train_loss: 0.016570
[040/01009] train_loss: 0.016706
[040/01059] train_loss: 0.016811
[040/01109] train_loss: 0.017144
[040/01159] train_loss: 0.017343
[040/01209] train_loss: 0.017002
[041/00033] train_loss: 0.019355
[041/00083] train_loss: 0.019048
[041/00133] train_loss: 0.018927
[041/00183] train_loss: 0.017667
[041/00233] train_loss: 0.017321
[041/00283] train_loss: 0.016578
[041/00333] train_loss: 0.016856
[041/00383] train_loss: 0.017789
[041/00433] train_loss: 0.016809
[041/00483] train_loss: 0.016819
[041/00533] train_loss: 0.015956
[041/00583] train_loss: 0.017424
[041/00633] train_loss: 0.016752
[041/00683] train_loss: 0.017015
[041/00733] train_loss: 0.016378
[041/00783] train_loss: 0.016595
[041/00833] train_loss: 0.016675
[041/00883] train_loss: 0.017331
[041/00933] train_loss: 0.016102
[041/00983] train_loss: 0.016431
[041/01033

...

[578/01021] train_loss: 0.008512
[578/01071] train_loss: 0.008928
[578/01121] train_loss: 0.009090
[578/01171] train_loss: 0.008236
[578/01221] train_loss: 0.008397
[579/00045] train_loss: 0.008447
[579/00095] train_loss: 0.008514
[579/00145] train_loss: 0.008723
[579/00195] train_loss: 0.008572
[579/00245] train_loss: 0.008833
[579/00295] train_loss: 0.008629
[579/00345] train_loss: 0.008550
[579/00395] train_loss: 0.009322
[579/00445] train_loss: 0.008346
[579/00495] train_loss: 0.008707
[579/00545] train_loss: 0.008401
[579/00595] train_loss: 0.009316
[579/00645] train_loss: 0.008371
[579/00695] train_loss: 0.008533
[579/00745] train_loss: 0.008020
[579/00795] train_loss: 0.008911
[579/00845] train_loss: 0.009168
[579/00895] train_loss: 0.009075
[579/00945] train_loss: 0.008873
[579/00995] train_loss: 0.008912
[579/01045] train_loss: 0.008632
[579/01095] train_loss: 0.009057
[579/01145] train_loss: 0.008854
[579/01195] train_loss: 0.008447
[580/00019] train_loss: 0.008162
[580/00069

[588/01211] train_loss: 0.008289
[589/00035] train_loss: 0.008148
[589/00085] train_loss: 0.008638
[589/00135] train_loss: 0.009395
[589/00185] train_loss: 0.009002
[589/00235] train_loss: 0.009086
[589/00285] train_loss: 0.008674
[589/00335] train_loss: 0.008765
[589/00385] train_loss: 0.008590
[589/00435] train_loss: 0.008641
[589/00485] train_loss: 0.008203
[589/00535] train_loss: 0.009179
[589/00585] train_loss: 0.008163
[589/00635] train_loss: 0.008758
[589/00685] train_loss: 0.008532
[589/00735] train_loss: 0.008829
[589/00785] train_loss: 0.009136
[589/00835] train_loss: 0.008218
[589/00885] train_loss: 0.009149
[589/00935] train_loss: 0.008477
[589/00985] train_loss: 0.008459
[589/01035] train_loss: 0.008821
[589/01085] train_loss: 0.008926
[589/01135] train_loss: 0.008688
[589/01185] train_loss: 0.008365
[590/00009] train_loss: 0.008050
[590/00059] train_loss: 0.009142
[590/00109] train_loss: 0.008321
[590/00159] train_loss: 0.008939
[590/00209] train_loss: 0.008690
[590/00259

[599/00175] train_loss: 0.008861
[599/00225] train_loss: 0.008323
[599/00275] train_loss: 0.008227
[599/00325] train_loss: 0.008997
[599/00375] train_loss: 0.008705
[599/00425] train_loss: 0.008689
[599/00475] train_loss: 0.008288
[599/00525] train_loss: 0.009460
[599/00575] train_loss: 0.008981
[599/00625] train_loss: 0.008755
[599/00675] train_loss: 0.009027
[599/00725] train_loss: 0.008802
[599/00775] train_loss: 0.008225
[599/00825] train_loss: 0.008669
[599/00875] train_loss: 0.008529
[599/00925] train_loss: 0.008403
[599/00975] train_loss: 0.008655
[599/01025] train_loss: 0.008320
[599/01075] train_loss: 0.008602
[599/01125] train_loss: 0.008599
[599/01175] train_loss: 0.008770
[599/01225] train_loss: 0.008577
[600/00049] train_loss: 0.008930
[600/00099] train_loss: 0.008912
[600/00149] train_loss: 0.008326
[600/00199] train_loss: 0.008999
[600/00249] train_loss: 0.008671
[600/00299] train_loss: 0.008800
[600/00349] train_loss: 0.008538
[600/00399] train_loss: 0.008933
[600/00449

[609/00365] train_loss: 0.008307
[609/00415] train_loss: 0.008228
[609/00465] train_loss: 0.009099
[609/00515] train_loss: 0.008312
[609/00565] train_loss: 0.009279
[609/00615] train_loss: 0.008758
[609/00665] train_loss: 0.008180
[609/00715] train_loss: 0.009129
[609/00765] train_loss: 0.008586
[609/00815] train_loss: 0.008662
[609/00865] train_loss: 0.008225
[609/00915] train_loss: 0.008702
[609/00965] train_loss: 0.008445
[609/01015] train_loss: 0.008697
[609/01065] train_loss: 0.008793
[609/01115] train_loss: 0.008912
[609/01165] train_loss: 0.008579
[609/01215] train_loss: 0.009200
[610/00039] train_loss: 0.008486
[610/00089] train_loss: 0.008844
[610/00139] train_loss: 0.008820
[610/00189] train_loss: 0.008432
[610/00239] train_loss: 0.008148
[610/00289] train_loss: 0.008707
[610/00339] train_loss: 0.008596
[610/00389] train_loss: 0.008477
[610/00439] train_loss: 0.008649
[610/00489] train_loss: 0.008570
[610/00539] train_loss: 0.008868
[610/00589] train_loss: 0.008036
[610/00639

[619/00555] train_loss: 0.008890
[619/00605] train_loss: 0.008494
[619/00655] train_loss: 0.009180
[619/00705] train_loss: 0.008708
[619/00755] train_loss: 0.009291
[619/00805] train_loss: 0.008289
[619/00855] train_loss: 0.008239
[619/00905] train_loss: 0.008342
[619/00955] train_loss: 0.008230
[619/01005] train_loss: 0.008733
[619/01055] train_loss: 0.008797
[619/01105] train_loss: 0.009060
[619/01155] train_loss: 0.008373
[619/01205] train_loss: 0.008451
[620/00029] train_loss: 0.008633
[620/00079] train_loss: 0.009065
[620/00129] train_loss: 0.008550
[620/00179] train_loss: 0.008448
[620/00229] train_loss: 0.009191
[620/00279] train_loss: 0.008349
[620/00329] train_loss: 0.008927
[620/00379] train_loss: 0.009046
[620/00429] train_loss: 0.009077
[620/00479] train_loss: 0.007983
[620/00529] train_loss: 0.008575
[620/00579] train_loss: 0.008182
[620/00629] train_loss: 0.007915
[620/00679] train_loss: 0.008675
[620/00729] train_loss: 0.008408
[620/00779] train_loss: 0.008920
[620/00829

### (f) Inference using the trained model on observed SDF values

Fill in the inference script `exercise_3/inference/infer_deepsdf.py`. Note that it's not simply a forward pass, but an optimization of the latent code such that we have lowest error on observed SDF values.

In [ ]:
from exercise_3.inference.infer_deepsdf import InferenceHandlerDeepSDF

device = torch.device('cuda:0')  # change this to cpu if you're not using a gpu

inference_handler = InferenceHandlerDeepSDF(256, "exercise_3/runs/3_2_deepsdf_generalization", device)

First, we try inference on a shape from validation set, for which we have a complete observation of sdf values. This is an easier problem as compared to shape completion,
since we have all the information already in the input.

Let's visualize the observations.

In [ ]:
# get observed data
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are very complete
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Reconstruction on these observations with the trained model:

In [ ]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

Next, we can try the shape completion task, i.e., inference on a shape from validation set, for which we do not have a complete observation of sdf values. The observed points are visualized below:

In [ ]:
# get observed data
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93_incomplete")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are incomplete
# making this is a shape completion task
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Shape completion using the trained model:

In [ ]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

### (g) Latent space interpolation

The latent space learned by DeepSDF is interpolatable, meaning that decoding latent codes from this space produced meaningful shapes. Given two latent codes, a linearly interpolatable latent space will decode
each of the intermediate codes to some valid shape. Let's see if this holds for our trained model.

We'll pick two shapes from the train set as visualized below.

In [ ]:
from exercise_3.data.shape_implicit import ShapeImplicit
from exercise_3.util.visualization import visualize_mesh

mesh = ShapeImplicit.get_mesh("494fe53da65650b8c358765b76c296")
print('GT Shape A')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

mesh = ShapeImplicit.get_mesh("5ca1ef55ff5f68501921e7a85cf9da35")
print('GT Shape B')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

Implement the missing parts in `exercise_3/inference/infer_deepsdf.py` such that it interpolates two given latent vectors, and run the code fragement below once done.

In [ ]:
from exercise_3.inference.infer_deepsdf import InferenceHandlerDeepSDF

inference_handler = InferenceHandlerDeepSDF(256, "exercise_3/runs/3_2_deepsdf_generalization", torch.device('cuda:0'))
# interpolate; also exports interpolated meshes to disk
inference_handler.interpolate('494fe53da65650b8c358765b76c296', '5ca1ef55ff5f68501921e7a85cf9da35', 60)

Visualize the interpolation below. If everything works out correctly, you should see a smooth transformation between the shapes, with all intermediate shapes being valid sofas.

In [ ]:
from exercise_3.util.mesh_collection_to_gif import  meshes_to_gif
from exercise_3.util.misc import show_gif

# create list of meshes (just exported) to be visualized
mesh_paths = sorted([x for x in Path("exercise_3/runs/3_2_deepsdf_generalization/interpolation").iterdir() if int(x.name.split('.')[0].split("_")[1]) == 0], key=lambda x: int(x.name.split('.')[0].split("_")[0]))
mesh_paths = mesh_paths + mesh_paths[::-1]

# create a visualization of the interpolation process
meshes_to_gif(mesh_paths, "exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif", 20)
show_gif("exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif")

## Submission

This is the end of exercise 3 🙂. Please create a zip containing all files we provided, everything you modified, your visualization images/gif (no need to submit generated OBJs), including your checkpoints. Name it with your matriculation number(s) as described in exercise 1. Make sure this notebook can be run without problems. Then, submit via Moodle.

**Note**: The maximum submission file size limit for Moodle is 100M. You do not need to submit your overfitting checkpoints; however, the generalization checkpoint will be >200M. The easiest way to still be able to submit that one is to split it with zip like this: `zip -s 100M model_best.ckpt.zip model_best.ckpt` which creates a `.zip` and a `.z01`. You can then submit both files alongside another zip containing all your code and outputs.

**Submission Deadline**: 09.06.2021, 23:55

## References

[1] Dai, Angela, Charles Ruizhongtai Qi, and Matthias Nießner. "Shape completion using 3d-encoder-predictor cnns and shape synthesis." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2017.

[2] Park, Jeong Joon, et al. "Deepsdf: Learning continuous signed distance functions for shape representation." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019.